# Домашняя работа №9

### Задания:
1) Взять код с урока и сделать для него "датасет"
2) Реализовать функцию у которой будут на выходе значения полученные нейросетью [не разобрался]

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


In [2]:
batch_size = 64
epochs = 30
latent_dim = 256
num_samples = 10000
# data_path = 'data/fra-eng/fra.txt'
data_path = 'data/rus-eng/rus.txt'

In [3]:
with open(data_path, 'r', encoding='utf-8') as file:
    lines = file.read().split('\n')

In [4]:
class TranslatorDataset(torch.utils.data.Dataset):
    
    def __init__(self, lines):
        global input_vocab2index, output_vocab2index
        self.lines = lines
        self.input_texts = []
        self.target_texts = []
        self.input_vocab = set()
        self.output_vocab = set()
        
        for line in self.lines[: min(num_samples, len(self.lines) - 1)]:
            input_text, target_text, _ = line.split('\t')
            target_text = '\t' + target_text + '\n'
            self.input_texts.append(input_text)
            for word in input_text.split():
                self.input_vocab.add(word.strip())
            self.target_texts.append(target_text)
            for word in target_text.split():
                self.output_vocab.add(word.strip())

        input_vocab2index = self.input_vocab2index = {word: i+2 for i, word in enumerate(self.input_vocab)}
        output_vocab2index = self.output_vocab2index = {word: i+2 for i, word in enumerate(self.output_vocab)}
        self.training_pairs = np.random.randint(0, len(self.input_texts), size=10000)
        
    def __len__(self):
        return len(self.training_pairs)
    
     
    def indexesFromSentence(self, sentence, vocab):
        return [vocab.get(word.strip(), 0) for word in sentence.split(' ')]
    
    def tensorFromSentence(self, sentence, vocab):
        indexes = self.indexesFromSentence(sentence, vocab)
        indexes.append(1)
        return torch.tensor(indexes, dtype=torch.long).view(-1, 1)
    
    
    def tensorsFromSent(self, input_sentences, output_sentences):
        input_tensor = self.tensorFromSentence(input_sentences, self.input_vocab2index)
        target_tensor = self.tensorFromSentence(output_sentences, self.output_vocab2index)
        return (input_tensor, target_tensor)
    
    
    def __getitem__(self, index):
        return self.tensorsFromSent(self.input_texts[self.training_pairs[index]], 
                                    self.target_texts[self.training_pairs[index]])

In [5]:
dataset = TranslatorDataset(lines)
loader = torch.utils.data.DataLoader(dataset,
                                     batch_size=128,
                                     shuffle=False,
                                     num_workers=0)

In [6]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input_, hidden):
        embedded = self.embedding(input_).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [7]:
class DecoderRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, input_size)
        
        
    def forward(self, input_, hidden, encoder_outputs):
        embedded = self.embedding(input_).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden.view(1, 1, 1, -1))
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)


class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=10):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        #output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [9]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=10):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[0]])

    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == 1:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

#### Pytroch ругается на не одинаковые тензоры

In [12]:
encoder = EncoderRNN(len(input_vocab2index)+2, 30)
attn_decoder1 = AttnDecoderRNN(30, len(output_vocab2index)+2, dropout_p=0.1)

encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = torch.optim.SGD(attn_decoder1.parameters(), lr=0.01)
criterion = nn.NLLLoss()

print_loss_total = 0
    
for i, data in enumerate(loader, 0):
    print(i)
    input_tensor, target_tensor = data[0], data[1]

    loss = train(input_tensor, target_tensor, encoder,
               attn_decoder1, encoder_optimizer, decoder_optimizer, criterion)
    print_loss_total += loss
    
    print_loss_avg = print_loss_total / 1
    print_loss_total = 0
    print(f'\r({i} {int(i / 10 * 100)}%) {print_loss_avg:.4f}', end='')

RuntimeError: stack expects each tensor to be equal size, but got [4, 1] at entry 0 and [3, 1] at entry 6

#### Поэтому обойдемся одним датасетом, без загрузчика

In [13]:
encoder = EncoderRNN(len(input_vocab2index)+2, 30)
attn_decoder1 = AttnDecoderRNN(30, len(output_vocab2index)+2, dropout_p=0.1)

encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = torch.optim.SGD(attn_decoder1.parameters(), lr=0.01)
criterion = nn.NLLLoss()

print_loss_total = 0
    
for i, (input_tensor, target_tensor) in enumerate(iter(dataset), 0):

    loss = train(input_tensor, target_tensor, encoder,
               attn_decoder1, encoder_optimizer, decoder_optimizer, criterion)
    print_loss_total += loss
    
    print_loss_avg = print_loss_total / 1
    print_loss_total = 0
    print(f'\r({i} {int(i / 10 * 100)}%) {print_loss_avg:.4f}', end='')

(9999 99990%) 3.3718